In [1]:
import pandas as pd                                         #Data scientist starter pack
import numpy as np
from matplotlib import pyplot as plt

import seaborn as sns                                       #More on plotting

import torch as trc                                         #To set PyTorch framework

import nltk                                                 #Natural language processing libraries
import spacy
import re                                                   #Regex tools

import random as rdm                                        #Simulation tools
import time                                                 #Ticking away the moments that make up a dull day

from tqdm import tqdm                                       #Te quiero desmasiado 

c:\Users\Leonardo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
news = pd.read_csv('datasets/RedditNews.csv')

In [15]:
combined = pd.read_csv('datasets/Combined_News_DJIA.csv')

Things to do:
- Explorative Data Analysis to evaluate datasets consistency
- Define function for text-preprocessing (stopwords, punctuation, stemming, tokenization)
- Start thinking about the feature engineering for the text mining 